In [ ]:
%run ../notebook_preamble.ipy

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from configparser import ConfigParser

In [ ]:
config_path = '/Users/grichardson/mysqldb_team.config'

## Load Data

In [ ]:
def get_engine(config_path, database="production", **engine_kwargs):
    '''Get a SQL alchemy engine from config'''
    cp = ConfigParser()
    cp.read(config_path)
    cp = cp["client"]
    url = URL(drivername="mysql+pymysql", database=database,
              username=cp["user"], host=cp["host"], password=cp["password"])
    return create_engine(url, **engine_kwargs)


chunksize = 1000

engine = get_engine(config_path)
engine.execution_options(stream_results=True)

# Projects
request = pd.read_sql_table('gtr_projects', engine, chunksize=chunksize)
projects = pd.concat(request)

In [ ]:
print(projects.shape)
projects.head()

In [ ]:
projects.to_csv(f'{data_path}/raw/gtr_projects.csv', index=False)

## EDA

In [ ]:
import pandas_profiling

### Profiling

In [ ]:
profile = projects.profile_report()

In [ ]:
profile

In [ ]:
profile.to_file(f'{project_dir}/reports/eda/gtr_projects_profile.html')

### Exploration

In [ ]:
fig, ax = plt.subplots()
projects.groupby('leadFunder')['id'].count().plot.bar(ax=ax)
ax.set_ylabel('Count')
plt.savefig(f'{project_dir}/reports/eda/gtr_projects_leadFunder_count_bar.png', dpi=300)
plt.show()

In [ ]:
abstract_lengths = projects['abstractText'].str.len()

fig, ax = plt.subplots()
ax.hist(abstract_lengths, bins=50)
ax.set_ylabel('Frequency')
ax.set_xlabel('Abstract Length')
plt.savefig(f'{project_dir}/reports/eda/gtr_projects_abstractText_length_hist.png', dpi=300)
plt.show()

- Looks like there's some kind of word limit.
- Can also see some peaks.
- Lengths may vary depending on funder. Let's look at this in more detail.

In [ ]:
abstract_lengths.hist??

In [ ]:
abstract_lengths.hist(
    by=projects['leadFunder'], density='normed',
    cumulative=True, sharex=True, sharey=True, histtype='step', bins=100, linewidth=2
)
plt.tight_layout()
plt.show()

In [ ]:
abstract_lengths[projects['leadFunder'] == 'ESRC'].hist(bins=np.linspace(0, 500))

It looks like there might be some boiler plate text. Let's have a look at that.

In [ ]:
for funder in projects['leadFunder'].unique():
    print(f'========={funder}==========')
    texts = projects[projects['leadFunder'] == funder]['abstractText'].value_counts()[:10]
    print(texts[texts.to_frame().index.str.len() > 100])

We will need to drop duplicates on `abstractText`, but we also need to get rid of rows with some specific abstract texts. We will create a custom data set with these and store it in `data/aux`.

We will drop any row containing boiler plate abstracts:

> Doctoral Training Partnerships: a range of postgraduate training is funded by the Research Councils. For information on current funding routes, see the common terminology at www.rcuk.ac.uk/StudentshipTerminology. Training grants may be to one organisation or to a consortia of research organisations. This portal will show the lead organisation only.

> Abstracts are not currently available in GtR for all funded research. This is normally because the abstract was not required at the time of proposal submission, but may be because it included sensitive information such as personal details.

We will also remove any abstracts with a length < 140 characters.

We will also drop duplicates of anything left over (retaining one copy).

There are no rows with missing abstracts or funders.

## Cleaning

Return a dataframe that only has the columns we need, and with rows dropped.

In [ ]:
with open(f'{data_path}/aux/gtr_projects_abstractText_drop.txt', 'r') as f:
    abstract_texts_drop = f.read().splitlines()

def clean(df):
    df = df[['abstractText', 'leadFunder']]
    df = df.drop_duplicates(subset='abstractText')
    df = df[df['abstractText'].str.len() > 100]
    df = df[~df['abstractText'].isin(abstract_texts_drop)]
    return df

In [ ]:
projects_clean = clean(projects)

In [ ]:
print(projects_clean.shape)
projects_clean.head()

## Text Pre-processing

In [ ]:
import re
import nltk
import string
from nltk.corpus import stopwords
from itertools import chain

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english') +
                 list(string.punctuation) +
                 ['\\n'] + ['quot'])

regex_str = ["http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|"
             r"[!*\(\),](?:%[0-9a-f][0-9a-f]))+",
             r"(?:\w+-\w+){2}",
             r"(?:\w+-\w+)",
             r"(?:\\\+n+)",
             r"(?:@[\w_]+)",
             "<[^>]+>",
             r"(?:\w+'\w)",
             r"(?:[\w_]+)",
             r"(?:\S)"
             ]

# Create the tokenizer which will be case insensitive and will ignore space.
tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')',
                       re.VERBOSE | re.IGNORECASE)


def tokenize_document(text, min_length=3, flatten=False):
    """Preprocess a whole raw document.
    Args:
        text (str): Raw string of text.
        min_length (int, optional): Minimum token length
        flatten (bool): Whether to flatten out sentences
    Returns:
        List: preprocessed and tokenized documents
    #UTILS
    """
    text = [clean_and_tokenize(sentence, min_length)
            for sentence in nltk.sent_tokenize(text)]
    if flatten:
        return list(chain(*text))
    else:
        return text


def clean_and_tokenize(text, min_length):
    """Preprocess a raw string/sentence of text.
    Args:
        text (str): Raw string of text.
        min_length (int): Minimum token length
    Returns:
        list of str: Preprocessed tokens.
    #UTILS
    """

    # Find tokens and lowercase
    tokens = tokens_re.findall(text)
    _tokens = [t.lower() for t in tokens]
    # Remove short tokens, stop words, tokens with digits, non-ascii chars
    filtered_tokens = [token.replace('-', '_') for token in _tokens
                       if len(token) >= min_length
                       and token not in stop_words
                       and not any(x in token for x in string.digits)
                       and any(x in token for x in string.ascii_lowercase)]
    return filtered_tokens

In [ ]:
min_length = 3
tokenized = projects_clean['abstractText'].apply(tokenize_document, min_length=min_length, flatten=True)